In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import wavfile
import os.path
import IPython.display
import seaborn as sns
import librosa
import librosa.display
import soundfile

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras import optimizers

import warnings

In [49]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
import re
import os
import wave
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydub import AudioSegment
from IPython.display import Audio, display
import librosa as lib
import librosa.display

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split

TensorFlow version: 2.12.0


In [56]:

PATH = "/content/drive/MyDrive/Colab Notebooks/Speech_Emotion/Crema"
AUDIO_PATH = "/content/drive/MyDrive/Colab Notebooks/Speech_Emotion/Crema/1001_DFA_ANG_XX.wav"

In [53]:
def zeroCrossingRate(audio):
  return lib.feature.zero_crossing_rate(audio)

In [ ]:
def energy(audio):
  # using a spectrogram will give a more accurate representation
  # of energy over time because its frames can be windowed
  S, phase = lib.magphase(lib.stft(audio))
  return lib.feature.rms(S=S).mean()

In [60]:
def melSpectrogram(audio, sr):
  mel_spectrogram = lib.feature.melspectrogram(y=audio, sr=sr, n_fft=200)
  log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
  return log_mel_spectrogram.mean()

In [ ]:
def chromaStft(audio, sr):
  stft = np.abs(librosa.stft(audio))
  return librosa.feature.chroma_stft(S=stft, sr=sr, n_fft=200).mean()

In [ ]:
def mfcc(data, sr):
  return librosa.feature.mfcc(y=data, sr=sr, n_fft=200).mean()

In [ ]:
def tonnetz(data, sr):
  return librosa.feature.tonnetz(y=data, sr=sr).mean();

In [ ]:
def playAudio(audio_file):
  audio = AudioSegment.from_wav(audio_file)
  # Play the audio
  audio.export('temp_audio.wav', format='wav')
  audio_data = open('temp_audio.wav', 'rb').read()
  display(Audio(audio_data))
  # Delete the temporary audio file
  os.remove('temp_audio.wav')

In [ ]:
def visualize_waveform(audio, sr):
    plt.figure(figsize=(12, 4))
    plt.plot(audio)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Waveform')
    plt.show()

In [66]:
def loadAndListenToAudiolib(dataset_path, class_name):
  D, Y = [], []
  # defining the regular expression
  audio_files = glob.glob(os.path.join(dataset_path, f"*{class_name}*.wav"))
  print(class_name)
  # play the audio
  '''playAudio(audio_files[0])'''
  # plot the spectrum
  '''audio, sr = lib.load(audio_files[0])'''
  '''visualize_waveform(audio, sr)'''
  # plotWaveform(audio_files[0], class_name)
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr)
    # chroma stft
    cs = chromaStft(audio, sr)
    mfc = mfcc(audio, sr)
    '''ton = tonnetz(audio, sr)'''
    combined_features = np.concatenate(([rms, mel_spec, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    print(len(combined_features))
    '''combined_features = [zcr, rms, mel_spec, cs, mfc, ton]'''
    # print(combined_features)
    '''D.append(combined_features)'''
    D.append(combined_features)
    Y.append(class_name)
  return D, Y

In [67]:
def loadDatalib(PATH):
  D, Y = [], []
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  for cls in classes: 
    d, y = loadAndListenToAudiolib(PATH, cls)
    # D = np.concatenate((D, d))
    D.extend(d)
    Y = np.concatenate((Y, y))
  return D, Y

In [68]:
D, Y = loadDatalib(PATH)

Streaming output truncated to the last 5000 lines.
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
DIS
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
304
3

In [70]:
D = np.array(D)
D.shape

(7442, 304)

In [71]:
Y.shape

(7442,)

In [72]:
df = pd.DataFrame(Y)

In [73]:
df.head()

,0
0,SAD
1,SAD
2,SAD
3,SAD
4,SAD


In [74]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(7442, 6)


In [75]:
Y = one_hot_labels

In [76]:
Y.shape

(7442, 6)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(D, Y, test_size=0.3, random_state=69, shuffle=True, stratify=Y)

In [78]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5209, 304)
(2233, 304)
(5209, 6)
(2233, 6)


In [79]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [80]:
print(X_train.shape, X_test.shape)

(5209, 304, 1) (2233, 304, 1)


In [ ]:
unicheck = np.unique(y_test, axis=0)

In [ ]:
unicheck

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]], dtype=uint8)

In [81]:
model = Sequential()
model.add(Conv1D(128, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D((1)))
model.add(Conv1D(256, 3, activation='relu'))
model.add(MaxPooling1D((1)))
model.add(Conv1D(512, 3, activation='relu'))
model.add(MaxPooling1D((1)))
model.add(Conv1D(1024, 3, activation='relu'))
model.add(MaxPooling1D((1)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 302, 128)          512       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 302, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 300, 256)          98560     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 300, 256)         0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 298, 512)          393728    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 298, 512)         0

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [82]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(X_test, y_test))

Epoch 1/50
41/41 [==============================] - 534s 13s/step - loss: 1.7670 - accuracy: 0.2102 - val_loss: 1.6914 - val_accuracy: 0.3283
Epoch 2/50
41/41 [==============================] - 537s 13s/step - loss: 1.6517 - accuracy: 0.3187 - val_loss: 1.5594 - val_accuracy: 0.3627
Epoch 3/50
 7/41 [====>.........................] - ETA: 6:16 - loss: 1.6216 - accuracy: 0.3214

KeyboardInterrupt: ignored